# Лекція 5: ООП в Python та Робота з файлами

**Курс**: Прикладна розробка програмного забезпечення на Python 2026
**Дата**: 26 лютого 2026

---

## Передумови

Ви вже знаєте:
- **Лекція 1**: Типи даних, змінні, f-рядки, venv, pip
- **Лекція 2**: Модель пам'яті, мутабельність, колекції, цикли та умови
- **Лекція 3**: Глибоке занурення в колекції, comprehensions, функції, args/kwargs
- **Лекція 4**: Lambda, map/filter/reduce, генератори, замикання, декоратори, модулі, винятки, type hints

## Цілі навчання

Після цієї лекції ви зможете:

1. **Визначати та використовувати класи** в Python: `__init__`, `self`, атрибути, методи
2. **Застосовувати 4 принципи ООП**: інкапсуляцію, успадкування, поліморфізм, абстракцію
3. **Використовувати Pythonic-патерни**: `@property`, `@dataclass`, `@classmethod`/`@staticmethod`, ABC
4. **Читати та записувати файли (files)**, серіалізувати об'єкти в JSON та CSV
5. **Побудувати міні-проєкт**: Контактна книга, що об'єднує ООП + серіалізацію

## Вступ

У Лекції 4 ми навчились писати функції, модулі та обробляти помилки.
Сьогодні зробимо великий крок: перейдемо від **процедурного програмування** до **об'єктно-орієнтованого**.

> 💡 **Ідея**: замість того, щоб тримати дані в словниках і передавати їх у купу функцій — ми об'єднаємо дані та поведінку в один об'єкт (object).

![OOP in Python](https://www.meme-arsenal.com/memes/13fc3f32be10c416a2c04e491f27001b.jpg)

---

## 1. Основи ООП

В цьому розділі ви зрозумієте, **навіщо** існують класи, і напишете свій перший клас з нуля.

### 1.1 Навіщо ООП?

Уявіть, що вам потрібно зберігати інформацію про книги в бібліотеці.
Спочатку використовуємо підхід зі словниками (dict) та функціями (functions):

In [3]:
# Підхід 1: Словники та функції (ПРОЦЕДУРНИЙ / PROCEDURAL)
# Дані та поведінка розкидані по різних місцях

def create_book(title, author, year):
    return {"title": title, "author": author, "year": year}

def describe_book(book):
    return f"{book['title']} ({book['year']}) — {book['author']}"

def is_classic(book):
    return book["year"] < 1980

# Використання
book1 = create_book("Володар Перснів", "Джон Рональд Руел Толкін", 1954)
book2 = create_book("Кобзар", "Тарас Шевченко", 1840)

print(describe_book(book1))
print(describe_book(book2))
print("Класика?", is_classic(book2))

# Проблема: дані (dict) і функції — окремо.
# Якщо структура зміниться — треба оновлювати всі функції.

Володар Перснів (1954) — Джон Рональд Руел Толкін
Кобзар (1840) — Тарас Шевченко
Класика? True


In [4]:
# Підхід 2: Клас (OBJECT-ORIENTED / ООП)
# Дані та поведінка — в одному місці

class Book:
    def __init__(self, title: str, author: str, year: int):
        # __init__ — конструктор (constructor), викликається при створенні об'єкта
        # self — посилання на поточний об'єкт (instance)
        self.title = title
        self.author = author
        self.year = year

    def describe(self) -> str:
        return f"{self.title} ({self.year}) — {self.author}"

    def is_classic(self) -> bool:
        return self.year < 1980


# Створення об'єктів (instances)
book1 = Book("Володар Перснів", "Джон Рональд Руел Толкін", 1954)
book2 = Book("Кобзар", "Тарас Шевченко", 1840)

print(book1.describe())
print(book2.describe())
print("Класика?", book2.is_classic())

# Перевага: дані та методи — разом.
# Легко додати новий метод, не ламаючи нічого.

Володар Перснів (1954) — Джон Рональд Руел Толкін
Кобзар (1840) — Тарас Шевченко
Класика? True


### 1.2 Синтаксис класу: `class`, `__init__`, `self`

| Ключове слово | Пояснення |
|---------------|-----------|
| `class`       | Оголошує новий клас (blueprint для об'єктів) |
| `__init__`    | Конструктор (constructor) — викликається при `MyClass(...)` |
| `self`        | Посилання (reference) на поточний об'єкт — як `this` у Java/C# |

> **Правило**: перший параметр методу завжди `self` — Python передає його автоматично.

In [5]:
# Мінімальний клас: Person
class Person:
    def __init__(self, name: str, age: int):
        self.name = name   # атрибут екземпляра (instance attribute)
        self.age = age

    def greet(self) -> str:
        return f"Привіт! Я {self.name}, мені {self.age} років."

    def is_adult(self) -> bool:
        return self.age >= 18


# Клас — це шаблон (blueprint)
print(type(Person))  # <class 'type'>

<class 'type'>


In [6]:
# Створення екземплярів (instances) та доступ до атрибутів

person1 = Person("Іван", 25)
person2 = Person("Оля", 16)

# Доступ до атрибутів через крапку
print(person1.name)       # Іван
print(person1.age)        # 25
print(person1.greet())    # Привіт! Я Іван, мені 25 років.
print(person2.is_adult()) # False

# Кожен об'єкт — незалежний
person1.age = 26           # змінюємо age тільки у person1
print(person1.age, person2.age)  # 26 16

Іван
25
Привіт! Я Іван, мені 25 років.
False
26 16


### 1.3 Атрибути екземпляра vs Атрибути класу (Instance vs Class Variables)

| Тип | Де визначається | Унікальний для... | Приклад |
|-----|-----------------|-------------------|---------|
| **Instance variable** | В `__init__` через `self.x = ...` | Кожного об'єкта | `self.name` |
| **Class variable** | В тілі класу (поза методами) | Усього класу (shared) | `species = "Homo sapiens"` |

> ⚠️ **Обережно**: якщо клас-атрибут — мутабельний (list, dict), зміна через один екземпляр впливає на всіх!

In [7]:

# Демонстрація: змінна класу (class variable) vs змінна екземпляра (instance variable)

class Counter:
    count = 0  # клас-атрибут: спільний для всіх екземплярів

    def __init__(self, name: str):
        Counter.count += 1       # збільшуємо загальний лічильник
        self.name = name         # атрибут екземпляра: унікальний
        self.value = 0           # атрибут екземпляра: унікальний


c1 = Counter("Alpha")
c2 = Counter("Beta")
c3 = Counter("Gamma")

print(f"Всього лічильників: {Counter.count}")  # 3 — спільне для всіх
print(f"c1.name = {c1.name}, c1.value = {c1.value}")
print(f"c2.name = {c2.name}, c2.value = {c2.value}")


Всього лічильників: 3
c1.name = Alpha, c1.value = 0
c2.name = Beta, c2.value = 0


In [8]:
# Мутація: клас-атрибут змінюється для всіх, instance-атрибут — тільки для одного

class Dog:
    species = "Canis lupus familiaris"  # клас-атрибут

    def __init__(self, name: str):
        self.name = name  # instance-атрибут

rex = Dog("Рекс")
luna = Dog("Луна")

print(f"До зміни: rex.species = {rex.species}")
print(f"До зміни: luna.species = {luna.species}")

# Змінюємо клас-атрибут через клас — зміниться у всіх
Dog.species = "Wolf-Dog Hybrid"
print(f"Після Dog.species: rex.species = {rex.species}")
print(f"Після Dog.species: luna.species = {luna.species}")

# Змінюємо через екземпляр — створюється instance-атрибут, клас НЕ змінюється
rex.species = "Poodle Mix"
print(f"Після rex.species: rex.species = {rex.species}")    # тільки рекс
print(f"Після rex.species: luna.species = {luna.species}")  # луна незмінна

До зміни: rex.species = Canis lupus familiaris
До зміни: luna.species = Canis lupus familiaris
Після Dog.species: rex.species = Wolf-Dog Hybrid
Після Dog.species: luna.species = Wolf-Dog Hybrid
Після rex.species: rex.species = Poodle Mix
Після rex.species: luna.species = Wolf-Dog Hybrid


---

## 2. Чотири Принципи ООП

| Принцип | Що означає | Python-механізм | Приклад |
|---------|------------|-----------------|---------|
| **Інкапсуляція** (Encapsulation) | Ховаємо деталі реалізації | `_attr`, `__attr`, `@property` | `BankAccount._balance` |
| **Успадкування** (Inheritance) | Клас отримує поведінку батька | `class Dog(Animal)`, `super()` | `Dog` наслідує `Animal` |
| **Поліморфізм** (Polymorphism) | Один інтерфейс — різна поведінка | Перевизначення методів (method overriding) | `Circle.area()` vs `Rectangle.area()` |
| **Абстракція** (Abstraction) | Контракт без реалізації | `abc.ABC`, `@abstractmethod` | `Shape` як абстрактний базовий клас |

---

### 2.1 Інкапсуляція

Python **не забороняє** доступ до атрибутів — це лише конвенція (convention):

| Синтаксис | Назва | Що означає |
|-----------|-------|-----------|
| `self.balance` | Публічний (public) | Доступний звідусіль |
| `self._balance` | Захищений (protected) | "Не чіпай без потреби" — конвенція |
| `self.__balance` | Приватний (private) | Name mangling → `_ClassName__balance` |

> **Ключова думка**: У Python немає справжнього `private`. Є лише домовленість (convention) між розробниками.

In [9]:
class BankAccount:
    def __init__(self, owner: str, balance: float, pin: str):
        self.owner = owner          # публічний (public): вільний доступ
        self._balance = balance     # захищений (protected): конвенція "не чіпай"
        self.__pin = pin            # приватний (private): name mangling

    def get_balance(self) -> float:
        return self._balance

    def deposit(self, amount: float) -> None:
        if amount > 0:
            self._balance += amount
            print(f"Поповнено: +{amount} грн. Баланс: {self._balance} грн")


account = BankAccount("Іван", 1000.0, "1234")
print(account.owner)          # OK — публічний
print(account._balance)       # Працює, але порушує конвенцію!
print(account.get_balance())  # Правильний спосіб

Іван
1000.0
1000.0


In [10]:
# Демонстрація name mangling (перекручення імен)

# Спроба отримати __pin напряму — викличе AttributeError!
try:
    print(account.__pin)
except AttributeError as e:
    print(f"Помилка: {e}")

# Але через name mangling — можна (просто незручно):
print(account._BankAccount__pin)  # '1234' — Python перетворює __pin у _BankAccount__pin

# Висновок: Python дає нам КОНВЕНЦІЮ, а не ЗАХИСТ.
# "Ми всі дорослі" — Python Zen

Помилка: 'BankAccount' object has no attribute '__pin'
1234


### 2.2 Успадкування

Успадкування дозволяє одному класу (дочірньому/subclass) **отримати** поведінку та атрибути іншого (батьківського/superclass).

- `super().__init__(...)` — викликає конструктор батьківського класу
- **MRO (Method Resolution Order)** — порядок, у якому Python шукає методи у ієрархії класів

In [12]:
# Просте успадкування (single inheritance)

class Animal:
    def __init__(self, name: str, sound: str):
        self.name = name
        self.sound = sound

    def speak(self) -> str:
        return f"{self.name} каже: {self.sound}!"

    def __repr__(self) -> str:
        return f"Animal(name={self.name!r})"


class Dog(Animal):
    def __init__(self, name: str, breed: str):
        super().__init__(name, sound="Гав")  # викликаємо Animal.__init__
        self.breed = breed

    def fetch(self) -> str:
        return f"{self.name} приніс м'яч!"

    def __repr__(self) -> str:
        return f"Dog(name={self.name!r}, breed={self.breed!r})"


class GuideDog(Dog):
    def __init__(self, name: str, breed: str, owner: str):
        super().__init__(name, breed)
        self.owner = owner

    def guide(self) -> str:
        return f"{self.name} веде {self.owner}"


rex = Dog("Рекс", "Вівчарка")
guide = GuideDog("Барон", "Лабрадор", "Іван")

print(rex.speak())       # успадковано від Animal
print(rex.fetch())       # власний метод Dog
print(guide.speak())     # успадковано від Animal через Dog
print(guide.guide())     # власний метод GuideDog

Рекс каже: Гав!
Рекс приніс м'яч!
Барон каже: Гав!
Барон веде Іван


In [13]:
# Множинне успадкування (multiple inheritance) та MRO (method resolution order)

class A:
    def greet(self):
        return "Привіт від A"

class B(A):
    def greet(self):
        return "Привіт від B"

class C(A):
    def greet(self):
        return "Привіт від C"

class D(B, C):
    pass  # D не визначає greet — Python шукає за MRO


obj = D()
print(obj.greet())  # "Привіт від B" — Python знаходить B першим за MRO

# Перегляд MRO (Method Resolution Order)
print("MRO для D:", [cls.__name__ for cls in D.__mro__])
# D → B → C → A → object

Привіт від B
MRO для D: ['D', 'B', 'C', 'A', 'object']


In [14]:
# Алмазна проблема (Diamond Problem) — Python вирішує її через C3 MRO

class Base:
    def hello(self):
        return "Base.hello"

class Left(Base):
    def hello(self):
        return "Left.hello"

class Right(Base):
    def hello(self):
        return "Right.hello"

class Diamond(Left, Right):
    pass


d = Diamond()
print(d.hello())  # Left.hello — MRO визначає порядок

# Python використовує C3 linearization (лінеаризацію C3)
# Порядок пошуку: Diamond → Left → Right → Base → object
for cls in Diamond.__mro__:
    print(f"  → {cls.__name__}")

# Схема:
#        Base
#       /    \
#     Left   Right
#       \    /
#       Diamond
#
# MRO: Diamond → Left → Right → Base → object

Left.hello
  → Diamond
  → Left
  → Right
  → Base
  → object


### 2.3 Поліморфізм

**Поліморфізм** означає: один інтерфейс (метод), різна поведінка — залежно від типу об'єкта.

У Python поліморфізм базується на **duck typing**:
> "Якщо воно ходить як качка і крякає як качка — це качка."
> Нас не цікавить тип об'єкта — лише чи є в нього потрібний метод.

In [15]:
import math

# Ієрархія фігур — демонстрація поліморфізму

class Shape:
    def area(self) -> float:
        raise NotImplementedError("Підкласи мають реалізувати area()")

    def describe(self) -> str:
        return f"{type(self).__name__}: площа = {self.area():.2f}"


class Circle(Shape):
    def __init__(self, radius: float):
        self.radius = radius

    def area(self) -> float:
        return math.pi * self.radius ** 2


class Rectangle(Shape):
    def __init__(self, width: float, height: float):
        self.width = width
        self.height = height

    def area(self) -> float:
        return self.width * self.height


class Triangle(Shape):
    def __init__(self, base: float, height: float):
        self.base = base
        self.height = height

    def area(self) -> float:
        return 0.5 * self.base * self.height


# Поліморфізм у дії: один цикл — різна поведінка
shapes = [Circle(5), Rectangle(4, 6), Triangle(3, 8)]

for shape in shapes:
    print(shape.describe())  # кожен shape.area() — своя реалізація

Circle: площа = 78.54
Rectangle: площа = 24.00
Triangle: площа = 12.00


### 2.4 Абстракція

**Абстракція** — це контракт (contract): "цей клас ЗОБОВ'ЯЗАНИЙ реалізувати певні методи".

У Python це реалізується через модуль `abc` (Abstract Base Classes):
- `abc.ABC` — базовий клас для абстрактних класів
- `@abstractmethod` — позначає метод, який ОБОВ'ЯЗКОВО треба перевизначити в підкласі

> **Аналогія**: `interface` у Java/C# — лише через Python-конвенцію.

In [16]:
from abc import ABC, abstractmethod
import math


class Shape(ABC):
    """Абстрактний базовий клас (Abstract Base Class) для фігур."""

    @abstractmethod
    def area(self) -> float:
        """Підклас ЗОБОВ'ЯЗАНИЙ реалізувати цей метод."""
        ...

    @abstractmethod
    def perimeter(self) -> float:
        ...

    def describe(self) -> str:
        # Конкретний метод — успадковується підкласами
        return f"{type(self).__name__}: площа={self.area():.2f}, периметр={self.perimeter():.2f}"


class Circle(Shape):
    def __init__(self, radius: float):
        self.radius = radius

    def area(self) -> float:
        return math.pi * self.radius ** 2

    def perimeter(self) -> float:
        return 2 * math.pi * self.radius


# Спроба створити екземпляр абстрактного класу — TypeError!
try:
    s = Shape()
except TypeError as e:
    print(f"Помилка: {e}")

# Конкретний підклас — OK
c = Circle(5)
print(c.describe())
# Далі ABC буде розглянуто детальніше в Розділі 3.4

Помилка: Can't instantiate abstract class Shape without an implementation for abstract methods 'area', 'perimeter'
Circle: площа=78.54, периметр=31.42


### 2.5 Магічні методи (Dunder / Magic Methods)

Python автоматично викликає **dunder-методи** (double underscore — подвійне підкреслення) у відповідь на певні операції:


**Магічні методи - прекрасна ілюстрація пайтонівського принципу "Duck Typing"**

| Метод | Коли викликається | Що повертати |
|-------|-------------------|--------------|
| `__repr__` | `repr(obj)`, в REPL | Детальний рядок для розробника |
| `__str__` | `str(obj)`, `print(obj)` | Зручний рядок для користувача |
| `__eq__` | `obj1 == obj2` | `bool` |
| `__len__` | `len(obj)` | `int` |
| `__hash__` | `hash(obj)`, dict/set | `int` (необхідний якщо `__eq__` визначено) |

> **Правило**: якщо ви визначили `__eq__`, визначте і `__hash__` (або встановіть `__hash__ = None` для unhashable).

In [18]:
class Point:
    def __init__(self, x: float, y: float):
        self.x = x
        self.y = y

    def __repr__(self) -> str:
        # repr() — для розробника, детально
        return f"Point(x={self.x}, y={self.y})"

    def __str__(self) -> str:
        # str() — для користувача, зручно
        return f"({self.x}, {self.y})"

    def __eq__(self, other: object) -> bool:
        # == порівнює за значенням, а не за посиланням
        if not isinstance(other, Point):
            return NotImplemented
        return self.x == other.x and self.y == other.y

    def __hash__(self) -> int:
        # потрібен, бо визначили __eq__
        return hash((self.x, self.y))

    def __len__(self) -> int:
        # len() — кількість координат
        return 2


p1 = Point(3, 4)
p2 = Point(3, 4)
p3 = Point(1, 2)

print(repr(p1))        # Point(x=3, y=4) — __repr__
print(str(p1))         # (3, 4) — __str__
print(p1 == p2)        # True — __eq__ (за значенням!)
print(p1 == p3)        # False
print(p1 is p2)        # False — різні об'єкти в пам'яті!
print(len(p1))         # 2 — __len__
print(hash(p1))        # число — __hash__

# Тепер Point можна використовувати як ключ у dict або в set
seen = {p1, p2, p3}
print(f"Унікальних точок: {len(seen)}")  # 2 (p1 == p2)

Point(x=3, y=4)
(3, 4)
True
False
False
2
1079245023883434373
Унікальних точок: 2


### 2.6 Композиція > Успадкування (Composition > Inheritance)

**Проблема глибокого успадкування**: зміна в базовому класі ламає всі підкласи ("крихкий базовий клас").

**Правило**: надавайте перевагу **"has-a"** (має) над **"is-a"** (є).
- `Dog` **is-a** `Animal` → успадкування
- `Car` **has-a** `Engine` → композиція (не `Car` **is-a** `Engine`)

In [ ]:
# ❌ ПОГАНО: глибока ієрархія успадкування

class Vehicle:
    def __init__(self, speed):
        self.speed = speed
    def move(self):
        return f"Рухається зі швидкістю {self.speed} км/год"

class LandVehicle(Vehicle):
    def drive(self):
        return "Їде по дорозі"

class Car(LandVehicle):
    def __init__(self, speed, brand):
        super().__init__(speed)
        self.brand = brand
    def honk(self):
        return f"{self.brand}: Бііп!"

class ElectricCar(Car):
    def __init__(self, speed, brand, battery_kw):
        super().__init__(speed, brand)
        self.battery_kw = battery_kw
    def charge(self):
        return f"Зарядка {self.battery_kw} кВт·год"

# Якщо Vehicle.move() зміниться — всі 4 класи можуть зламатись!
ec = ElectricCar(200, "Tesla", 75)
print(ec.move(), "|", ec.drive(), "|", ec.honk(), "|", ec.charge())

Рухається зі швидкістю 200 км/год | Їде по дорозі | Tesla: Бііп! | Зарядка 75 кВт·год


**А тепер уявіть, що клас при визначення класу Car розробники не продумали різні види двигунів**

In [21]:
class Car(LandVehicle):
    def __init__(self, speed, brand):
        super().__init__(speed)
        self.brand = brand
        self.motor = "внутрішнього згоряння"
    def __str__(self):
        return f"{self.brand} (motor: {self.motor})"
    def honk(self):
        return f"{self.brand}: Бііп!"

class ElectricCar(Car):
    def __init__(self, speed, brand, battery_kw):
        super().__init__(speed, brand)
        self.battery_kw = battery_kw
    def charge(self):
        return f"Зарядка {self.battery_kw} кВт·год"

print(ElectricCar(150, "Toyota", 50))

Toyota (motor: внутрішнього згоряння)


In [27]:
# ДОБРЕ: композиція (composition) — "has-a"

class Engine:
    def __init__(self, power_kw: int, en_type: str = "внутрішнього згоряння"):
        self.power_kw = power_kw
        self.en_type = en_type
    def __str__(self):
        return f"Двигун: {self.power_kw} кВт, тип: {self.en_type}"
    def start(self) -> str:
        return f"Двигун {self.power_kw} кВт запущено"
    
class CarV2:
    def __init__(self, brand: str, speed: int, engine_power_kw: int):
        self.brand = brand
        self.speed = speed
        # Composition: Car HAS-A Engine
        self.engine = Engine(power_kw=engine_power_kw)
    
    def __str__(self):
        return f"{self.brand} (двигун: {self.engine})"

    def drive(self) -> str:
        return f"{self.brand} їде зі швидкістю {self.speed} км/год"

    def honk(self) -> str:
        return f"{self.brand}: Бііп!"

class Battery:
    def __init__(self, capacity_kwh: float):
        self.capacity_kwh = capacity_kwh
    
    def __str__(self):
        return f"Батарея: {self.capacity_kwh} кВт·год"

    def charge(self) -> str:
        return f"Заряджається {self.capacity_kwh} кВт·год"

class ElectricCarV2:
    def __init__(self, brand: str, speed: int, battery_kwh: float, engine_power_kw: int = 150):
        self.brand = brand
        self.speed = speed
        # Composition: ElectricCarV2 HAS-A Engine і HAS-A Battery
        self.engine = Engine(power_kw=engine_power_kw)
        self.battery = Battery(capacity_kwh=battery_kwh)
    
    def __str__(self):
        return f"{self.brand} (двигун: {self.engine}, батарея: {self.battery})"

    def drive(self) -> str:
        return f"{self.brand} їде зі швидкістю {self.speed} км/год"

    def honk(self) -> str:
        return f"{self.brand}: Бііп!"


# Зміни в Engine або Battery не впливають на ElectricCarV2 та CarV2 напряму
car2 = CarV2("Toyota Camry", 180, engine_power_kw=150)
ec2 = ElectricCarV2("Tesla Model 3", 250, battery_kwh=75, engine_power_kw=200)
print(car2)
print(ec2)

# Favour composition over inheritance — GoF Design Patterns, 1994

Toyota Camry (двигун: Двигун: 150 кВт, тип: внутрішнього згоряння)
Tesla Model 3 (двигун: Двигун: 200 кВт, тип: внутрішнього згоряння, батарея: Батарея: 75 кВт·год)


***Коротко зазирнемо у світ проектування коду***

**Dependency inversion** (літера D з SOLID)

У попередніх прикладах CarV2 та ElectricCarV2 створюють інстанси двигуна **всередині** свого конструктора, відповідно, якщо ми хочемо досягти варіативності у тому, які двигуни стоять у різних автомобілях, ми повинні додавати багато аргументів до конструктора даних класів (як от потужність мотора) і ми розуміємо, що в подальшому це стане некерованим.

**Принцип інверсії залежностей (Dependency Inversion Principle)** каже:

> Модулі високого рівня (Car) **не повинні** залежати від модулів низького рівня (конкретний Engine).
> Обидва повинні залежати від **абстракцій** (інтерфейсів).

**Простими словами**: замість того, щоб Car сам створював собі двигун, ми **передаємо** (inject) готовий двигун ззовні. Car знає лише про абстракцію `PowerSource`, а не про конкретну реалізацію.

```
❌ БЕЗ DIP:  Car → створює Engine всередині (жорстка залежність)
✅ З DIP:    Car → залежить від абстракції PowerSource ← Engine реалізує
```

In [28]:
from abc import ABC, abstractmethod

# Абстракція: контракт для будь-якого джерела енергії
class PowerSource(ABC):
    @abstractmethod
    def start(self) -> str:
        ...

    @abstractmethod
    def __str__(self) -> str:
        ...

# Конкретна реалізація 1: бензиновий двигун
class GasolineEngine(PowerSource):
    def __init__(self, power_kw: int):
        self.power_kw = power_kw

    def start(self) -> str:
        return f"Бензиновий двигун {self.power_kw} кВт: *ВРРРРМ*"

    def __str__(self) -> str:
        return f"Бензиновий {self.power_kw} кВт"

# Конкретна реалізація 2: електричний мотор + батарея
class ElectricDrive(PowerSource):
    def __init__(self, power_kw: int, battery_kwh: float):
        self.power_kw = power_kw
        self.battery_kwh = battery_kwh

    def start(self) -> str:
        return f"Електромотор {self.power_kw} кВт (батарея {self.battery_kwh} кВт·год): *тиша...*"

    def __str__(self) -> str:
        return f"Електро {self.power_kw} кВт, батарея {self.battery_kwh} кВт·год"


# Модуль високого рівня: Car залежить ТІЛЬКИ від абстракції PowerSource
class CarV3:
    def __init__(self, brand: str, speed: int, power_source: PowerSource):
        self.brand = brand
        self.speed = speed
        self.power_source = power_source   # Dependency Injection!

    def __str__(self) -> str:
        return f"{self.brand} ({self.power_source})"

    def start(self) -> str:
        return self.power_source.start()

    def drive(self) -> str:
        return f"{self.brand} їде зі швидкістю {self.speed} км/год"


# --- Демонстрація ---

# Створюємо джерела енергії ОКРЕМО
gas = GasolineEngine(power_kw=150)
electric = ElectricDrive(power_kw=200, battery_kwh=75)

# Ін'єктуємо (inject) їх у автомобілі
toyota = CarV3("Toyota Camry", 180, power_source=gas)
tesla  = CarV3("Tesla Model 3", 250, power_source=electric)

print(toyota)
print(toyota.start())
print()
print(tesla)
print(tesla.start())

# CarV3 не знає, чи в нього бензиновий чи електричний мотор —
# він просто викликає power_source.start() і все працює!
print("\n--- Легко замінити двигун ---")
hybrid_engine = GasolineEngine(power_kw=100)
toyota_hybrid = CarV3("Toyota Prius", 160, power_source=hybrid_engine)
print(toyota_hybrid)
print(toyota_hybrid.start())

Toyota Camry (Бензиновий 150 кВт)
Бензиновий двигун 150 кВт: *ВРРРРМ*

Tesla Model 3 (Електро 200 кВт, батарея 75 кВт·год)
Електромотор 200 кВт (батарея 75 кВт·год): *тиша...*

--- Легко замінити двигун ---
Toyota Prius (Бензиновий 100 кВт)
Бензиновий двигун 100 кВт: *ВРРРРМ*


---

## 3. Pythonic ООП Патерни

Цей розділ — про те, що відрізняє **Python OOP** від Java/C# OOP.
Ви побачите, чому Python-код зазвичай коротший, виразніший та "чистіший".

### 3.1 `@property` — Контрольований доступ до атрибутів

У Java/C# для захисту атрибутів пишуть `getBalance()`/`setBalance()`.
У Python — використовуємо `@property`:

```python
# ❌ Java-стиль (анти-патерн у Python)
account.set_balance(-100)   # незручно

# ✅ Python-стиль
account.balance = -100      # виглядає як атрибут, але виконує валідацію!
```

In [29]:
class BankAccount:
    def __init__(self, owner: str, initial_balance: float):
        self.owner = owner
        self._balance = initial_balance  # зберігаємо в _balance (protected)

    @property
    def balance(self) -> float:
        """Геттер (getter): читаємо _balance через balance."""
        return self._balance

    @balance.setter
    def balance(self, value: float) -> None:
        """Сеттер (setter): валідація перед записом."""
        if value < 0:
            raise ValueError(f"Баланс не може бути від'ємним! Отримано: {value}")
        self._balance = value

    def deposit(self, amount: float) -> None:
        self.balance += amount  # використовує setter з валідацією!

    def __repr__(self) -> str:
        return f"BankAccount(owner={self.owner!r}, balance={self._balance})"


acc = BankAccount("Оля", 500.0)
print(acc.balance)    # 500.0 — через геттер (getter)

acc.balance = 1000    # через сеттер (setter) — OK
print(acc.balance)    # 1000

acc.deposit(250)
print(acc)            # BankAccount(owner='Оля', balance=1250)

# Спроба встановити від'ємний баланс
try:
    acc.balance = -100
except ValueError as e:
    print(f"Помилка: {e}")

500.0
1000
BankAccount(owner='Оля', balance=1250)
Помилка: Баланс не може бути від'ємним! Отримано: -100


### 3.2 `@classmethod` та `@staticmethod`

| | Instance method | `@classmethod` | `@staticmethod` |
|--|----------------|----------------|-----------------|
| **Перший аргумент** | `self` (екземпляр) | `cls` (клас) | Немає |
| **Доступ до екземпляра** | ✅ | ❌ | ❌ |
| **Доступ до класу** | через `self.__class__` | ✅ | ❌ |
| **Типовий сценарій** | Поведінка об'єкта | Альтернативний конструктор | Утилітарна функція |

> **Правило**: якщо метод не використовує `self` і не потребує `cls` — зробіть його `@staticmethod`.
> Якщо потрібен `cls` (наприклад, для `return cls(...)`) — `@classmethod`.

In [30]:
class Contact:
    def __init__(self, name: str, phone: str, email: str = ""):
        self.name = name
        self.phone = phone
        self.email = email

    # Instance method: використовує self
    def display(self) -> str:
        return f"📇 {self.name} | {self.phone}" + (f" | {self.email}" if self.email else "")

    # @classmethod: альтернативний конструктор (factory method)
    @classmethod
    def from_dict(cls, data: dict) -> "Contact":
        """Створює Contact зі словника (dict)."""
        return cls(
            name=data["name"],
            phone=data["phone"],
            email=data.get("email", "")
        )

    # @staticmethod: утилітарна функція, що логічно належить класу
    @staticmethod
    def validate_phone(phone: str) -> bool:
        """Перевіряє, чи починається номер з +380."""
        return phone.startswith("+380") and len(phone) == 13


# Instance method
c1 = Contact("Іван Петренко", "+380501234567", "ivan@example.com")
print(c1.display())

# @classmethod — альтернативний конструктор
data = {"name": "Оля Коваль", "phone": "+380671234567"}
c2 = Contact.from_dict(data)
print(c2.display())

# @staticmethod — без екземпляра
print(Contact.validate_phone("+380501234567"))  # True
print(Contact.validate_phone("0501234567"))      # False

📇 Іван Петренко | +380501234567 | ivan@example.com
📇 Оля Коваль | +380671234567
True
False


### 3.3 `@dataclass` — Менше бойлерплейту

`@dataclass` (з Python 3.7) автоматично генерує `__init__`, `__repr__` та `__eq__` за описом полів.

In [32]:
# Порівняння: вручну vs @dataclass

# Вручну — багато повторювального коду
class ProductManual:
    def __init__(self, name: str, price: float, quantity: int = 0):
        self.name = name
        self.price = price
        self.quantity = quantity

    def __repr__(self):
        return f"Product(name={self.name!r}, price={self.price}, quantity={self.quantity})"

    def __eq__(self, other):
        if not isinstance(other, ProductManual):
            return NotImplemented
        return (self.name, self.price, self.quantity) == (other.name, other.price, other.quantity)


# З @dataclass — те саме, але коротше
from dataclasses import dataclass, field

@dataclass
class Product:
    name: str
    price: float
    quantity: int = 0  # значення за замовчуванням (default)


# Поведінка ідентична!
p1 = Product("Ноутбук", 25000.0, 5)
p2 = Product("Ноутбук", 25000.0, 5)
p3 = ProductManual("Ноутбук", 25000.0, 5)

print(p1)           # Product(name='Ноутбук', price=25000.0, quantity=5)
print(p1 == p2)     # True — __eq__ згенеровано автоматично
print(p1 == p3)     # False — різні класи

Product(name='Ноутбук', price=25000.0, quantity=5)
True
False


In [33]:
from dataclasses import dataclass, field
from dataclasses import FrozenInstanceError

# Проблема: мутабельний дефолт (MUTABLE DEFAULT FOOTGUN)
# ❌ Так не можна:
# @dataclass
# class Bag:
#     items: list = []  # ВСІ екземпляри ділять ОДИН список!

# ✅ Правильно: field(default_factory=list)
@dataclass
class Bag:
    items: list = field(default_factory=list)  # кожен екземпляр — свій список
    name: str = "Bag"

bag1 = Bag()
bag2 = Bag()
bag1.items.append("яблуко")
print(f"bag1: {bag1.items}")  # ['яблуко']
print(f"bag2: {bag2.items}")  # [] — НЕ поділяється!


# frozen=True — незмінний (immutable) dataclass
@dataclass(frozen=True)
class FrozenPoint:
    x: float
    y: float

fp = FrozenPoint(1.0, 2.0)
print(fp)  # FrozenPoint(x=1.0, y=2.0)

try:
    fp.x = 10.0  # FrozenInstanceError!
except Exception as e:
    print(f"Помилка: {type(e).__name__}: {e}")

bag1: ['яблуко']
bag2: []
FrozenPoint(x=1.0, y=2.0)
Помилка: FrozenInstanceError: cannot assign to field 'x'


### 3.4 ABC — Повторення та Поглиблення

> Ми вже бачили `ABC` у Розділі 2.4. Тут — кілька додаткових можливостей.

`@abstractmethod` можна поєднувати з `@property` — для абстрактних властивостей:

In [34]:
from abc import ABC, abstractmethod

class Drawable(ABC):
    @property
    @abstractmethod
    def color(self) -> str:
        """Абстрактна властивість (abstract property)."""
        ...

    @abstractmethod
    def draw(self) -> str:
        ...

class RedCircle(Drawable):
    @property
    def color(self) -> str:
        return "red"

    def draw(self) -> str:
        return f"Малюю {self.color} коло"


rc = RedCircle()
print(rc.draw())     # Малюю red коло
print(rc.color)      # red

# ABC — це Python-аналог interface у Java та interface у C#
# Різниця: Python перевіряє лише при створенні екземпляра, не при компіляції

Малюю red коло
red


### 3.5 Python vs Java/C#

| Особливість | Java/C# | Python |
|-------------|---------|--------|
| **Модифікатори доступу** | `private`, `protected`, `public` | Конвенція: `_attr`, `__attr` |
| **Інтерфейси (interfaces)** | `interface` ключове слово | `abc.ABC` + `@abstractmethod` |
| **Геттери/сеттери** | Явні методи `getX()`/`setX()` | `@property` + `@x.setter` |
| **Перевантаження операторів** | Обмежено / явні методи | Dunder-методи (`__add__`, `__eq__`, …) |
| **Типізація (type system)** | Номінальна (явні типи) | Duck typing + type hints (необов'язково) |
| **`null`** | `null` (Java) / `null` (C#) | `None` |
| **Статичні методи** | `static` модифікатор | `@staticmethod` декоратор |

> **Головна ідея**: Python обирає **конвенцію над примусом (convention over enforcement)**.
> Розробники домовляються між собою, а не покладаються на компілятор.

### 3.6 Цікаві Особливості Python

Кілька прийомів, які роблять Python OOP ще потужнішим (і інколи дивним):

In [35]:
# __slots__: фіксуємо список атрибутів екземпляра
# Переваги: менше пам'яті, швидший доступ до атрибутів

class PointWithSlots:
    __slots__ = ("x", "y")  # дозволяємо ТІЛЬКИ ці атрибути

    def __init__(self, x: float, y: float):
        self.x = x
        self.y = y

p = PointWithSlots(1.0, 2.0)
print(p.x, p.y)  # OK

# Спроба додати новий атрибут — AttributeError!
try:
    p.z = 3.0
except AttributeError as e:
    print(f"Помилка: {e}")

# Без __slots__: кожен об'єкт має __dict__ (хеш-таблиця)
# З __slots__: замість __dict__ — фіксований масив (економія пам'яті ~30-50%)
try:
    print(p.__dict__)
except AttributeError:
    print("Немає __dict__ — __slots__ замінив його!")

1.0 2.0
Помилка: 'PointWithSlots' object has no attribute 'z' and no __dict__ for setting new attributes
Немає __dict__ — __slots__ замінив його!


In [36]:
# Небезпека: мутабельний аргумент за замовчуванням у __init__

# ❌ ПОГАНО: всі екземпляри ділять ОДИН список!
class BagBroken:
    def __init__(self, items=[]):  # ДЕФЕКТ! items — один об'єкт для всіх
        self.items = items

b1 = BagBroken()
b2 = BagBroken()
b1.items.append("яблуко")
print(f"b2.items = {b2.items}")  # ['яблуко'] — b2 теж змінився!

print()

# ✅ ДОБРЕ: None як sentinel, новий список у кожному виклику
class BagFixed:
    def __init__(self, items=None):
        self.items = items if items is not None else []

b3 = BagFixed()
b4 = BagFixed()
b3.items.append("груша")
print(f"b4.items = {b4.items}")  # [] — не змінився

# Ця ж проблема: чому @dataclass використовує field(default_factory=list)

b2.items = ['яблуко']

b4.items = []


In [37]:
# Протокол контекстного менеджера (Context Manager Protocol)
# __enter__ та __exit__ — це ті самі методи, що використовує `with open()`!

import time

class Timer:
    """Контекстний менеджер для вимірювання часу."""

    def __enter__(self):
        self.start = time.perf_counter()
        print("Таймер запущено...")
        return self  # повертається як `as` значення

    def __exit__(self, exc_type, exc_val, exc_tb):
        elapsed = time.perf_counter() - self.start
        print(f"Час виконання: {elapsed:.4f} секунд")
        return False  # False = не пригнічуємо виключення


with Timer() as t:
    total = sum(range(1_000_000))

print(f"Результат: {total}")

# Це той самий механізм, що і:
# with open("file.txt") as f:
#     ...
# open() повертає об'єкт з __enter__/__exit__!

Таймер запущено...
Час виконання: 0.0202 секунд
Результат: 499999500000


---

## 4. Робота з файлами (File I/O)

Тепер навчимося зберігати дані на диску: читати та записувати файли безпечно.

### 4.1 `open()` та Контекстні менеджери (Context Managers)

Правильний спосіб відкривати файли — через `with`:

```python
# ✅ Контекстний менеджер: файл ЗАВЖДИ буде закрито
with open("file.txt", "w", encoding="utf-8") as f:
    f.write("Текст")
# f.close() викликається автоматично — навіть при виключенні!

# ❌ Без контекстного менеджера — ризик незакритого файлу:
f = open("file.txt", "w")
f.write("Текст")
f.close()  # якщо тут виняток — close() ніколи не викличеться
```

| Режим | Значення |
|-------|---------|
| `"r"` | Читання (read) — за замовчуванням |
| `"w"` | Запис (write) — перезаписує файл |
| `"a"` | Дозапис (append) — додає в кінець |
| `"r+"` | Читання + запис |
| `"rb"` | Читання у бінарному режимі (binary) |

In [38]:
# Запис файлу (writing)
with open("test.txt", "w", encoding="utf-8") as f:
    f.write("Привіт, Іванко!\n")
    f.write("Це тест кодування UTF-8.\n")
    f.write("Слава Україні! 🇺🇦\n")

print("Файл test.txt створено")

Файл test.txt створено


In [39]:
# Читання файлу (reading)
with open("test.txt", "r", encoding="utf-8") as f:
    content = f.read()

print("Вміст файлу:")
print(content)
print(f"Довжина: {len(content)} символів")

# Читання рядок за рядком (line by line)
print("\nПорядкове читання:")
with open("test.txt", "r", encoding="utf-8") as f:
    for i, line in enumerate(f, start=1):
        print(f"  Рядок {i}: {line.rstrip()}")

Вміст файлу:
Привіт, Іванко!
Це тест кодування UTF-8.
Слава Україні! 🇺🇦

Довжина: 59 символів

Порядкове читання:
  Рядок 1: Привіт, Іванко!
  Рядок 2: Це тест кодування UTF-8.
  Рядок 3: Слава Україні! 🇺🇦


### 4.2 Кодування (Encoding)

> **Правило**: **завжди** вказуйте `encoding='utf-8'` у всіх файлових операціях.

Чому це важливо:
- На **Windows** стандартне кодування — `cp1251` (кирилиця) або `cp1252` (Western European)
- На **Linux/Mac** — `utf-8`
- Без явного `encoding` ваш код буде непередбачуваним на різних платформах
- **UTF-8** — єдиний правильний вибір для багатомовних додатків

In [41]:
from pathlib import Path

# Демонстрація UTF-8 round-trip з українськими символами
ukrainian_text = "Привіт! Я вивчаю Python. Їжак їсть їжу. Є, і, ї, ю, я."

# Записуємо з utf-8
with open("encoding_test.txt", "w", encoding="utf-8") as f:
    f.write(ukrainian_text)

# Читаємо з utf-8
with open("encoding_test.txt", "r", encoding="utf-8") as f:
    read_back = f.read()

# Перевірка: round-trip повинен повернути точно той самий текст
assert read_back == ukrainian_text, "Кодування зламало текст!"
print("UTF-8 round-trip успішний!")
print(f"Оригінал: {ukrainian_text}")
print(f"Прочитано: {read_back}")
print(f"Ідентично: {read_back == ukrainian_text}")

UTF-8 round-trip успішний!
Оригінал: Привіт! Я вивчаю Python. Їжак їсть їжу. Є, і, ї, ю, я.
Прочитано: Привіт! Я вивчаю Python. Їжак їсть їжу. Є, і, ї, ю, я.
Ідентично: True


### 4.3 `pathlib.Path` — Сучасний API для шляхів

`pathlib` (Python 3.4+) — рекомендований спосіб роботи з файловими шляхами.
Замість конкатенації рядків: `"data" + "/" + "file.txt"` — використовуйте оператор `/`.

In [42]:
from pathlib import Path

# Створення шляху через оператор /
data_dir = Path("data")
contacts_file = data_dir / "contacts.json"

print(f"Шлях: {contacts_file}")
print(f"Батьківська директорія: {contacts_file.parent}")
print(f"Розширення: {contacts_file.suffix}")
print(f"Ім'я без розширення: {contacts_file.stem}")
print(f"Існує? {contacts_file.exists()}")

# Створення директорії (якщо не існує)
data_dir.mkdir(exist_ok=True)
print(f"Директорія '{data_dir}' готова")

# Запис та читання через pathlib
contacts_file.write_text('{"contacts": []}', encoding="utf-8")
content = contacts_file.read_text(encoding="utf-8")
print(f"Вміст: {content}")

# Перелік файлів у директорії
print("\nФайли в поточній директорії:")
for f in Path(".").iterdir():
    if f.is_file() and f.suffix in (".txt", ".json", ".csv"):
        print(f"  {f.name}")

# os.path існує для зворотної сумісності (legacy code):
# import os; os.path.join("data", "contacts.json") — старий спосіб

Шлях: data\contacts.json
Батьківська директорія: data
Розширення: .json
Ім'я без розширення: contacts
Існує? False
Директорія 'data' готова
Вміст: {"contacts": []}

Файли в поточній директорії:
  encoding_test.txt
  test.txt


---

## 5. JSON — Серіалізація Даних (Data Serialization)

**JSON (JavaScript Object Notation)** — текстовий формат для обміну та зберігання даних.
Він читабельний для людини і підтримується практично всіма мовами.

### 5.1 Основи JSON в Python

| Функція | Вхід → Вихід | Коли використовувати |
|---------|-------------|----------------------|
| `json.dumps(obj)` | Python → рядок JSON | API-відповіді, логування |
| `json.loads(s)` | рядок JSON → Python | Парсинг API-відповідей |
| `json.dump(obj, f)` | Python → файл JSON | Зберігання на диску |
| `json.load(f)` | файл JSON → Python | Завантаження з диску |

In [43]:
import json

# json.dumps() та json.loads() — рядковий (string-based) варіант
data = {
    "name": "Іван",
    "age": 25,
    "skills": ["Python", "SQL", "Git"],
    "active": True,
    "score": None
}

# Python → JSON рядок
json_string = json.dumps(data, ensure_ascii=False, indent=2)
print("JSON рядок:")
print(json_string)

# JSON рядок → Python
restored = json.loads(json_string)
print(f"\nВідновлено: {restored}")
print(f"Ідентично: {restored == data}")

JSON рядок:
{
  "name": "Іван",
  "age": 25,
  "skills": [
    "Python",
    "SQL",
    "Git"
  ],
  "active": true,
  "score": null
}

Відновлено: {'name': 'Іван', 'age': 25, 'skills': ['Python', 'SQL', 'Git'], 'active': True, 'score': None}
Ідентично: True


In [44]:
import json
from pathlib import Path

# json.dump() та json.load() — файловий (file-based) варіант
contacts = [
    {"name": "Іван Петренко",  "phone": "+380501234567", "email": "ivan@example.com"},
    {"name": "Оля Коваль",     "phone": "+380671234567", "email": ""},
    {"name": "Михайло Бойко",  "phone": "+380931234567", "email": "mykhailo@example.com"},
]

# Запис у файл
output_file = Path("contacts_demo.json")
with open(output_file, "w", encoding="utf-8") as f:
    json.dump({"contacts": contacts}, f, ensure_ascii=False, indent=2)

print(f"Збережено {len(contacts)} контактів у {output_file}")

# Читання з файлу
with open(output_file, "r", encoding="utf-8") as f:
    loaded = json.load(f)

print(f"\nЗавантажено {len(loaded['contacts'])} контактів:")
for c in loaded["contacts"]:
    print(f"  {c['name']} — {c['phone']}")

Збережено 3 контактів у contacts_demo.json

Завантажено 3 контактів:
  Іван Петренко — +380501234567
  Оля Коваль — +380671234567
  Михайло Бойко — +380931234567


### 5.2 Мапінг Python типів на JSON типи
JSON підтримує **обмежений набір типів**:

| Python тип | JSON тип | Приклад |
|-----------|---------|---------|
| `dict` | `object {}` | `{"key": "value"}` |
| `list` | `array []` | `[1, 2, 3]` |
| `str` | `string ""` | `"hello"` |
| `int`, `float` | `number` | `42`, `3.14` |
| `True` / `False` | `true` / `false` | `true` |
| `None` | `null` | `null` |
| `datetime`, `set`, клас | ❌ Не підтримується | — |

> **Висновок**: перед серіалізацією складні об'єкти потрібно **конвертувати** у примітиви JSON.
> Бібліотеки як **Pydantic** (Лекція 6) роблять це автоматично.

### 5.3 Не-серіалізовані Типи
Що робити, якщо JSON не знає, як серіалізувати ваш тип?

In [45]:
import json
from datetime import datetime

# Проблема: datetime не підтримується JSON
try:
    json.dumps({"created_at": datetime.now()})
except TypeError as e:
    print(f"Помилка: {e}")

# Рішення 1: default=str — просто і швидко (але втрачаємо тип!)
result = json.dumps({"created_at": datetime.now()}, default=str)
print(f"\ndefault=str: {result}")

# Рішення 2: Власна конвертація перед серіалізацією
data = {
    "event": "lecture",
    "created_at": datetime.now().isoformat(),  # конвертуємо явно
    "tags": list({"python", "oop", "files"})   # set → list
}
print(f"\nЯвна конвертація: {json.dumps(data, ensure_ascii=False, indent=2)}")

# Рішення 3 (advanced): JSONEncoder — для великих проєктів
class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime):
            return obj.isoformat()
        return super().default(obj)

result3 = json.dumps({"ts": datetime.now()}, cls=CustomEncoder)
print(f"\nCustomEncoder: {result3}")

Помилка: Object of type datetime is not JSON serializable

default=str: {"created_at": "2026-02-26 13:52:19.899230"}

Явна конвертація: {
  "event": "lecture",
  "created_at": "2026-02-26T13:52:19.899370",
  "tags": [
    "oop",
    "files",
    "python"
  ]
}

CustomEncoder: {"ts": "2026-02-26T13:52:19.899643"}


---

## 6. CSV — Табличні Дані

**CSV (Comma-Separated Values)** — простий текстовий формат для таблиць.
Широко використовується у data science, Excel-файлах та базах даних.

In [46]:
import csv
from pathlib import Path

# Спочатку створимо sample.csv для демонстрації
sample_data = [
    ["name", "phone", "email"],
    ["Іван Петренко",  "+380501234567", "ivan@example.com"],
    ["Оля Коваль",     "+380671234567", "ola@example.com"],
    ["Михайло Бойко",  "+380931234567", "mykhailo@example.com"],
]

with open("sample.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(sample_data)

print("Файл sample.csv створено")

Файл sample.csv створено


### 6.1 Читання CSV (Reading CSV)

| Клас | Повертає | Коли використовувати |
|------|---------|----------------------|
| `csv.reader` | `list` рядків (list of lists) | Коли заголовок не потрібен |
| `csv.DictReader` | `dict` по заголовку (list of dicts) | Коли потрібні іменовані колонки |

In [47]:
import csv

print("=== csv.reader (список рядків / list of lists) ===")
with open("sample.csv", "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    for row in reader:
        print(row)  # кожен рядок — список рядків

print()

print("=== csv.DictReader (словники / dicts) ===")
with open("sample.csv", "r", encoding="utf-8") as f:
    reader = csv.DictReader(f)  # перший рядок — заголовок
    for row in reader:
        print(dict(row))  # кожен рядок — dict з іменованими ключами

# DictReader значно зручніший:
print()
print("Зручний доступ за назвою колонки:")
with open("sample.csv", "r", encoding="utf-8") as f:
    for contact in csv.DictReader(f):
        print(f"  {contact['name']} → {contact['phone']}")

=== csv.reader (список рядків / list of lists) ===
['name', 'phone', 'email']
['Іван Петренко', '+380501234567', 'ivan@example.com']
['Оля Коваль', '+380671234567', 'ola@example.com']
['Михайло Бойко', '+380931234567', 'mykhailo@example.com']

=== csv.DictReader (словники / dicts) ===
{'name': 'Іван Петренко', 'phone': '+380501234567', 'email': 'ivan@example.com'}
{'name': 'Оля Коваль', 'phone': '+380671234567', 'email': 'ola@example.com'}
{'name': 'Михайло Бойко', 'phone': '+380931234567', 'email': 'mykhailo@example.com'}

Зручний доступ за назвою колонки:
  Іван Петренко → +380501234567
  Оля Коваль → +380671234567
  Михайло Бойко → +380931234567


### 6.2 Запис CSV (Writing CSV)

> **Важливо**: використовуйте `newline=""` при відкритті файлу для csv.writer — інакше будуть зайві пусті рядки.

In [48]:
import csv
from pathlib import Path

# csv.DictWriter — зручний запис з заголовком
new_contacts = [
    {"name": "Тетяна Лисенко", "phone": "+380441234567", "email": "t.lysenko@example.com"},
    {"name": "Андрій Мельник", "phone": "+380961234567", "email": ""},
]

output_path = Path("new_contacts.csv")

with open(output_path, "w", newline="", encoding="utf-8") as f:
    fieldnames = ["name", "phone", "email"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)

    writer.writeheader()           # записуємо рядок заголовка
    writer.writerows(new_contacts)  # записуємо всі рядки

print(f"Записано {len(new_contacts)} контактів у {output_path}")

# Перевірка: читаємо назад
print("\nВміст нового файлу:")
with open(output_path, "r", encoding="utf-8") as f:
    print(f.read())

Записано 2 контактів у new_contacts.csv

Вміст нового файлу:
name,phone,email
Тетяна Лисенко,+380441234567,t.lysenko@example.com
Андрій Мельник,+380961234567,



### 6.3 Роздільники (Delimiters)

CSV не завжди означає "кома". Поширені варіанти:
- `;` — стандарт у деяких країнах та Excel (особливо в Україні)
- `\t` — TSV (Tab-Separated Values) — часто у data science

In [49]:
import csv

# Запис з роздільником "крапка з комою"
data = [
    {"місто": "Київ", "населення": 2952301},
    {"місто": "Харків", "населення": 1421125},
    {"місто": "Одеса", "населення": 993120},
]

with open("cities.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["місто", "населення"], delimiter=";")
    writer.writeheader()
    writer.writerows(data)

print("Файл cities.csv (роздільник ';'):")
with open("cities.csv", "r", encoding="utf-8") as f:
    print(f.read())

# Читання з роздільником ';'
print("Читання через DictReader(delimiter=';'):")
with open("cities.csv", "r", encoding="utf-8") as f:
    for row in csv.DictReader(f, delimiter=";"):
        print(f"  {row['місто']}: {int(row['населення']):,} мешканців")

# Для TSV (tab-separated): delimiter='\t'

Файл cities.csv (роздільник ';'):
місто;населення
Київ;2952301
Харків;1421125
Одеса;993120

Читання через DictReader(delimiter=';'):
  Київ: 2,952,301 мешканців
  Харків: 1,421,125 мешканців
  Одеса: 993,120 мешканців


---

## 7. Практичні Вправи

Час практикуватись! Спробуйте вирішити вправи самостійно, перш ніж переглядати рішення.

In [ ]:
# Вправа 1: BankAccount з @property
# ═══════════════════════════════════

# Реалізуйте клас BankAccount з такими вимогами:
# 1. __init__(self, owner: str, initial_balance: float)
# 2. @property balance — повертає поточний баланс
# 3. @balance.setter — відхиляє від'ємні значення (raise ValueError)
# 4. deposit(amount: float) — поповнення (перевіряє amount > 0)
# 5. withdraw(amount: float) — зняття (перевіряє amount <= balance)
# 6. __repr__ — зручне відображення

class BankAccount:
    # Вправа 1: Ваш код тут
    pass


# Тест (розкоментуйте після реалізації):
# acc = BankAccount("Іван", 1000.0)
# print(acc)                    # BankAccount(owner='Іван', balance=1000.0)
# acc.deposit(500)
# print(acc.balance)            # 1500.0
# acc.withdraw(200)
# print(acc.balance)            # 1300.0
# try:
#     acc.balance = -100        # ValueError!
# except ValueError as e:
#     print(e)
# try:
#     acc.withdraw(9999)        # ValueError!
# except ValueError as e:
#     print(e)

<details>
<summary>Рішення (клікніть щоб побачити)</summary>

```python
class BankAccount:
    def __init__(self, owner: str, initial_balance: float):
        self.owner = owner
        self._balance = initial_balance

    @property
    def balance(self) -> float:
        return self._balance

    @balance.setter
    def balance(self, value: float) -> None:
        if value < 0:
            raise ValueError(f"Баланс не може бути від'ємним! Отримано: {value}")
        self._balance = value

    def deposit(self, amount: float) -> None:
        if amount <= 0:
            raise ValueError("Сума поповнення має бути більше нуля")
        self.balance += amount
        print(f"✅ Поповнено +{amount} грн. Баланс: {self.balance} грн")

    def withdraw(self, amount: float) -> None:
        if amount > self.balance:
            raise ValueError(f"Недостатньо коштів. Баланс: {self.balance}, запит: {amount}")
        self.balance -= amount
        print(f"💸 Знято {amount} грн. Баланс: {self.balance} грн")

    def __repr__(self) -> str:
        return f"BankAccount(owner={self.owner!r}, balance={self._balance})"
```

</details>

In [ ]:
# Вправа 2: Ієрархія Фігур з ABC
# ═══════════════════════════════════

# Реалізуйте:
# 1. Абстрактний клас Shape(ABC) з методами area() і perimeter()
# 2. Клас Circle(Shape) з полем radius
# 3. Клас Rectangle(Shape) з полями width і height
# Обидва класи мають реалізувати area() і perimeter()

from abc import ABC, abstractmethod
import math


class Shape(ABC):
    # Вправа 2: Ваш код тут
    pass


class Circle(Shape):
    # Вправа 2: Ваш код тут
    pass


class Rectangle(Shape):
    # Вправа 2: Ваш код тут
    pass


# Тест:
# shapes = [Circle(5), Rectangle(3, 4)]
# for s in shapes:
#     print(f"{type(s).__name__}: площа={s.area():.2f}, периметр={s.perimeter():.2f}")

<details>
<summary>Рішення (клікніть щоб побачити)</summary>

```python
from abc import ABC, abstractmethod
import math


class Shape(ABC):
    @abstractmethod
    def area(self) -> float:
        ...

    @abstractmethod
    def perimeter(self) -> float:
        ...

    def describe(self) -> str:
        return f"{type(self).__name__}: площа={self.area():.2f}, периметр={self.perimeter():.2f}"


class Circle(Shape):
    def __init__(self, radius: float):
        self.radius = radius

    def area(self) -> float:
        return math.pi * self.radius ** 2

    def perimeter(self) -> float:
        return 2 * math.pi * self.radius


class Rectangle(Shape):
    def __init__(self, width: float, height: float):
        self.width = width
        self.height = height

    def area(self) -> float:
        return self.width * self.height

    def perimeter(self) -> float:
        return 2 * (self.width + self.height)


# Демонстрація поліморфізму
shapes = [Circle(5), Rectangle(3, 4)]
for s in shapes:
    print(s.describe())
```

</details>

---

## 8. Міні-проєкт: Контактна книга

**Мета**: побудуємо мінімальну Контактну книгу, яка об'єднує:
- `@dataclass` — для моделювання контакту (Contact)
- Клас `ContactBook` — для управління колекцією
- JSON — для збереження/завантаження даних на диск

**4 кроки**: Крок 1 → модель даних | Крок 2 → контейнер | Крок 3 → серіалізація | Крок 4 → демо

In [ ]:
# ════════════════════════════════════════════════════
# Крок 1: Модель даних — @dataclass Contact
# ════════════════════════════════════════════════════
from dataclasses import dataclass, field

@dataclass
class Contact:
    # Крок 1: Реалізовано — заповніть поля
    name: str          # обов'язкове поле
    phone: str         # обов'язкове поле
    email: str = ""    # необов'язкове поле (за замовчуванням пустий рядок)

    def display(self) -> str:
        parts = [f"📇 {self.name}", self.phone]
        if self.email:
            parts.append(self.email)
        return " | ".join(parts)


# Приклад використання:
c1 = Contact("Іван Петренко", "+380501234567", "ivan@example.com")
c2 = Contact("Оля Коваль", "+380671234567")

print(c1)           # Contact(name='Іван Петренко', ...) — __repr__ автоматичний
print(c1.display())
print(c2.display())

In [ ]:
# ════════════════════════════════════════════════════
# Крок 2: Контейнер — клас ContactBook
# ════════════════════════════════════════════════════
from __future__ import annotations

class ContactBook:
    def __init__(self):
        self._contacts: list[Contact] = []  # приватний список контактів

    def add(self, contact: Contact) -> None:
        """Додати контакт до книги."""
        self._contacts.append(contact)

    def search(self, name: str) -> list[Contact]:
        """Знайти контакти за частковим збігом імені (частково нечутливий до регістру)."""
        name_lower = name.lower()
        return [c for c in self._contacts if name_lower in c.name.lower()]

    def remove(self, name: str) -> bool:
        """Видалити перший контакт з точним збігом імені. Повертає True якщо знайдено."""
        for i, c in enumerate(self._contacts):
            if c.name == name:
                del self._contacts[i]
                return True
        return False

    def __len__(self) -> int:
        return len(self._contacts)

    def __repr__(self) -> str:
        return f"ContactBook({len(self)} контактів)"


# Перевірка
book = ContactBook()
book.add(Contact("Іван Петренко", "+380501234567", "ivan@example.com"))
book.add(Contact("Оля Коваль", "+380671234567"))

print(book)                          # ContactBook(2 контактів)
print(book.search("іван"))           # [Contact(...)]
print(book.remove("Оля Коваль"))     # True
print(len(book))                     # 1

In [ ]:
# ════════════════════════════════════════════════════
# Крок 3: Серіалізація — JSON збереження/завантаження
# ════════════════════════════════════════════════════
import json
from pathlib import Path

# Додаємо методи до ContactBook (через monkey-patching для демонстрації)
# У реальному коді ці методи будуть в тілі класу

def save_to_json(self, path: str | Path) -> None:
    """Зберегти книгу в JSON файл."""
    data = {
        "contacts": [
            {"name": c.name, "phone": c.phone, "email": c.email}
            for c in self._contacts
        ]
    }
    Path(path).write_text(
        json.dumps(data, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )
    print(f"✅ Збережено {len(self)} контактів у {path}")

@classmethod
def load_from_json(cls, path: str | Path) -> "ContactBook":
    """Альтернативний конструктор: завантажити ContactBook з JSON файлу."""
    data = json.loads(Path(path).read_text(encoding="utf-8"))
    book = cls()
    for item in data["contacts"]:
        book.add(Contact(**item))  # Contact(**{"name": ..., "phone": ..., "email": ...})
    print(f"✅ Завантажено {len(book)} контактів з {path}")
    return book

# "Монтуємо" методи до класу
ContactBook.save_to_json = save_to_json
ContactBook.load_from_json = load_from_json

print("Методи save_to_json та load_from_json готові!")

In [ ]:
# ════════════════════════════════════════════════════
# Крок 4: Демонстрація — повний round-trip
# ════════════════════════════════════════════════════

# 1. Створюємо книгу та додаємо контакти
book = ContactBook()
book.add(Contact("Іван Петренко",  "+380501234567", "ivan@example.com"))
book.add(Contact("Оля Коваль",     "+380671234567", "ola@example.com"))
book.add(Contact("Михайло Бойко",  "+380931234567", "mykhailo@example.com"))

print(f"Створено: {book}")

# 2. Зберігаємо на диск
book.save_to_json("contacts.json")

# 3. Завантажуємо з диску у новий об'єкт
loaded_book = ContactBook.load_from_json("contacts.json")
print(f"Завантажено: {loaded_book}")

# 4. Пошук контакту
found = loaded_book.search("іван")
print(f"\nРезультат пошуку 'іван': {len(found)} знайдено")
for c in found:
    print(f"  → {c.display()}")

# 5. Видалення та перевірка
removed = loaded_book.remove("Оля Коваль")
print(f"\nВидалено 'Оля Коваль': {removed}")
print(f"Залишилось: {len(loaded_book)} контактів")

<details>
<summary>Повне рішення (клікніть щоб побачити)</summary>

```python
import json
from dataclasses import dataclass
from pathlib import Path


@dataclass
class Contact:
    name: str
    phone: str
    email: str = ""

    def display(self) -> str:
        parts = [f"📇 {self.name}", self.phone]
        if self.email:
            parts.append(self.email)
        return " | ".join(parts)


class ContactBook:
    def __init__(self):
        self._contacts: list[Contact] = []

    def add(self, contact: Contact) -> None:
        self._contacts.append(contact)

    def search(self, name: str) -> list[Contact]:
        name_lower = name.lower()
        return [c for c in self._contacts if name_lower in c.name.lower()]

    def remove(self, name: str) -> bool:
        for i, c in enumerate(self._contacts):
            if c.name == name:
                del self._contacts[i]
                return True
        return False

    def save_to_json(self, path: str | Path) -> None:
        data = {
            "contacts": [
                {"name": c.name, "phone": c.phone, "email": c.email}
                for c in self._contacts
            ]
        }
        Path(path).write_text(
            json.dumps(data, ensure_ascii=False, indent=2),
            encoding="utf-8"
        )

    @classmethod
    def load_from_json(cls, path: str | Path) -> "ContactBook":
        data = json.loads(Path(path).read_text(encoding="utf-8"))
        book = cls()
        for item in data["contacts"]:
            book.add(Contact(**item))
        return book

    def __len__(self) -> int:
        return len(self._contacts)

    def __repr__(self) -> str:
        return f"ContactBook({len(self)} контактів)"
```

</details>

---

## Підсумок

### Що ми вивчили сьогодні:

**1. Основи ООП** — Розділ 1:
- Клас (class) як шаблон для об'єктів; `__init__`, `self`
- Атрибути екземпляра (instance) vs атрибути класу (class variables)

**2. Чотири принципи ООП** — Розділ 2:
- **Інкапсуляція**: `_attr`, `__attr`, name mangling (Python не примушує, лише конвенція)
- **Успадкування**: `super().__init__()`, MRO, множинне успадкування
- **Поліморфізм**: перевизначення методів, duck typing
- **Абстракція**: `abc.ABC`, `@abstractmethod`
- **Dunder-методи**: `__repr__`, `__str__`, `__eq__`, `__len__`, `__hash__`
- **Композиція > Успадкування**: "has-a" > "is-a" для гнучкого дизайну

**3. Pythonic-патерни** — Розділ 3:
- `@property` / `@x.setter` — контрольований доступ без гетерів/сетерів
- `@classmethod` — альтернативні конструктори; `@staticmethod` — утиліти
- `@dataclass` — автогенерація `__init__`, `__repr__`, `__eq__`; `frozen=True`
- `__slots__`, mutable default footgun, контекстний менеджер `__enter__`/`__exit__`

**4. Робота з файлами** — Розділ 4:
- `open()` + `with` — контекстний менеджер, автоматичне закриття
- `encoding='utf-8'` — обов'язково для українського тексту
- `pathlib.Path` — сучасний API шляхів з оператором `/`

**5. JSON** — Розділ 5:
- `json.dumps/loads` (рядки) та `json.dump/load` (файли)
- JSON-типи: лише dict, list, str, number, bool, null
- `default=str` для не-серіалізовних типів

**6. CSV** — Розділ 6:
- `csv.reader` (списки) та `csv.DictReader` (словники)
- `csv.DictWriter` + `writeheader()` + `writerows()`
- `delimiter=';'` для нестандартних роздільників

---

## Що далі?

### Лекція 6: REST + FastAPI Fundamentals

Ваш клас `Contact` з сьогоднішнього проєкту — це **концептуальна основа** для Pydantic-моделей у FastAPI!

У наступній лекції ми:
- **HTTP-основи**: методи (GET, POST, PUT, DELETE), статус-коди (status codes), заголовки
- **REST**: принципи та маппінг CRUD-операцій
- **FastAPI**: `app`, `router`, endpoints (ендпоінти)
- **Pydantic**: `BaseModel` для валідації запитів/відповідей — це `@dataclass` на стероїдах!
- **Auto docs**: Swagger / OpenAPI з коробки
- **Перший проєкт**: API для управління нотатками (notes) з in-memory зберіганням

> 💡 Підказка: `ContactBook` стане `NoteRepository`, а `Contact` стане `Note` у FastAPI-додатку!

### Домашнє завдання — Розширення міні-проєкту

**Завдання 1 — CSV-експорт**:
Додайте до `ContactBook` метод `export_to_csv(self, path: str | Path) -> None`,
який зберігає всі контакти у CSV-файл з колонками `name`, `phone`, `email`.

**Завдання 2 — Frozen Contact**:
Зробіть `Contact` незмінним: `@dataclass(frozen=True)`.
Оновіть `ContactBook.add()` та `load_from_json()` — що зміниться?

**Завдання 3 — Валідація email**:
Додайте `@staticmethod validate_email(email: str) -> bool` до `Contact`,
який перевіряє наявність символу `@` та крапки після нього.
Виклик: `Contact.validate_email("ivan@example.com")` → `True`.

---

## Джерела

### Офіційна документація
- [Python — Classes](https://docs.python.org/3/tutorial/classes.html) — Офіційний туторіал по класах
- [dataclasses](https://docs.python.org/3/library/dataclasses.html) — Документація @dataclass
- [abc — Abstract Base Classes](https://docs.python.org/3/library/abc.html) — Модуль ABC
- [json](https://docs.python.org/3/library/json.html) — Модуль JSON
- [csv](https://docs.python.org/3/library/csv.html) — Модуль CSV
- [pathlib](https://docs.python.org/3/library/pathlib.html) — Об'єктно-орієнтовані шляхи файлової системи

### Туторіали
- [Real Python — OOP in Python](https://realpython.com/python3-object-oriented-programming/) — Ґрунтовне введення в ООП
- [Real Python — Python Dataclasses](https://realpython.com/python-data-classes/) — Повний гід по @dataclass
- [Real Python — pathlib](https://realpython.com/python-pathlib/) — Робота з pathlib.Path
- [Real Python — Working With JSON Data](https://realpython.com/python-json/) — JSON в Python

### Поглиблене вивчення
- [Fluent Python (2nd Ed.) — Chapters 11–14](https://www.oreilly.com/library/view/fluent-python-2nd/9781492056348/) — ООП у глибині
- [Python Tricks: A Buffet of Awesome Python Features](https://realpython.com/products/python-tricks-book/) — OOP-розділ
- [Medium - Abstract Classes and Abstract Methods in Python](https://medium.com/@prashampahadiya9228/abstract-classes-and-abstract-methods-in-python-e632ea34bc79)
- [Mastering Dependency Inversion in Python Coding](https://arjancodes.com/blog/dependency-inversion-principle-in-python-programming/)
- [Medium - Dependency Injection Vs Dependency Inversion Vs Inversion of Control](https://medium.com/ssense-tech/dependency-injection-vs-dependency-inversion-vs-inversion-of-control-lets-set-the-record-straight-5dc818dc32d1)